# SARIMAX model
Google Trendsより，“大根”の検索に関する季節性を調べる。このデータの取得法は書籍，または，データファイルを参照されたい。<br>
このデータに対して，SARIMAX modelを適用する。<br>

Seasonal AutoRegressive Integrated Moving Average with eXogenous model  

SARIMAのモデル構造，引数（パラメータ）は次を参照：statsmodels.tsa.statespace.sarimax.SARIMAX  
http://www.statsmodels.org/dev/generated/statsmodels.tsa.statespace.sarimax.SARIMAX.html

引数の説明<br>
https://www.salesanalytics.co.jp/datascience/datascience087/#ARIMASARIMA<br>
非季節性
AR：p=1
階差：d=1
MA：q=1
季節性
季節AR：P=0
季節階差：D=1
季節MA：Q=0

In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import statsmodels.api as sm
import pmdarima as pm

from sklearn.model_selection import train_test_split

#from statsmodels.graphics.tsaplots import plot_predict

from scipy import stats

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
np.random.seed(123)
df = pd.read_csv('dataRadish.csv', index_col='Date', parse_dates=True, comment='#')

# split df into training and test data.
df_train, df_test = train_test_split(df, test_size=0.15, shuffle=False)
N = len(df_train)
print(len(df), N, N/(len(df)))

fig, ax = plt.subplots(figsize=(12,3))
df_train.plot(marker='o', markersize=3, ax=ax, color='k')
df_test.plot(ax=ax, color='b')

ax.grid()
#plt.show()

In [ ]:
print(df_train.head(10))

In [ ]:
acf = sm.tsa.stattools.acf(df, nlags=60)
fig, ax = plt.subplots(figsize=(6,3))
plt.plot(acf, marker='o')
plt.xlabel('lag')
plt.ylabel('acf')
plt.grid()

plt.show()

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12,3))
df_train.plot(ax=ax[0], marker='o', markersize=3, color='k')
df_test.plot(ax=ax[0], marker='o', markersize=5, color='dimgray', linestyle='dashed')

ax[0].grid()

acf = sm.tsa.stattools.acf(df, nlags=30)
ax[1].plot(acf, c='k', marker='o')
ax[1].set_xlabel('lag')
ax[1].set_ylabel('acf')
ax[1].grid()

#plt.savefig('fig_SARIMAX_Radish_Data.pdf', bbox_inches='tight')
plt.show()

## 自動構築（auto_arima関数）
https://www.salesanalytics.co.jp/datascience/datascience087/#ARIMASARIMA

pmdarimaのインストール, conda install pmdarima,  or pip install pmdarima<br>
参照
- http://alkaline-ml.com/pmdarima/
- https://www.salesanalytics.co.jp/datascience/datascience064/
- https://www.salesanalytics.co.jp/datascience/datascience087/

In [ ]:
sarima_model = pm.arima.auto_arima(df_train, 
                            seasonal=True,
                            m=12,
#                            m=52,
                            trace=True,
                            stepwise=True,
#                            n_jobs=-1,
                            maxiter=50)

In [ ]:
sarima_model.summary()

In [ ]:
# トレーニングデータの期間の予測値
train_pred = sarima_model.predict_in_sample()

# テストデータの期間の予測値, 予測区間
test_pred, test_pred_ci = sarima_model.predict(n_periods=df_test.shape[0], return_conf_int=True)

In [ ]:
df_test.shape[0]

In [ ]:
fig, ax = plt.subplots(figsize=(10,3))

ax.plot(df_train.index, df_train, label='actual train', c='k')
ax.plot(df_test.index, df_test, label='actual test', c='dimgray', linestyle='dashed')

#nstart=12
#ax.scatter(train_pred[nstart:].index, train_pred[nstart:], label='predict', color='k')
ax.scatter(train_pred.index, train_pred, label='predict', color='k')
ax.scatter(test_pred.index, test_pred, label='forecast', color='gray')

#x_axis = np.arange(preds.shape[0])
ax.fill_between(df_test.index, test_pred_ci[:,0], test_pred_ci[:,1], color='gray', alpha=0.3)
ax.grid()
plt.legend()
#plt.savefig('fig_SARIMAX_Radish_Data_Predict_Forecast.pdf', bbox_inches='tight')
plt.show()

In [ ]:
train_pred.head(15)